# Simple LlamaIndex RAG model

In [2]:
#some installs

# %pip install --upgrade llama-index
# %pip install --upgrade llama-index-vector-stores-chroma
# %pip install --upgrade llama-index-embeddings-openai
# %pip install --upgrade llama-index-llms-openai
# %pip install --upgrade llama-index-readers-file

## Basic RAG example 

In [3]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# initialize the LLM
llm = OpenAI(model="gpt-4.1-2025-04-14")
 
# initialize the embedding
embed_model = OpenAIEmbedding(model="text-embedding-3-small")


from llama_index.core import Settings
 
# global settings, this sets the LLM and embedding model for the entire program and the following code will use these models in the background
Settings.llm = llm
Settings.embed_model = embed_model


In [4]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

# load documents
data = SimpleDirectoryReader(input_dir="Data/").load_data()
 
# indexing documents using vector store 
# this is the most basic way to index documents and is just stored in memory, no Vecotor DB setup to store, so it is not be saved or reused when the program is closed
#I can show a vector store example later, this is just a simple example to get you something working as soon as possible
index = VectorStoreIndex.from_documents(data, show_progress=True)

# converting vector store to query engine
query_engine = index.as_query_engine(similarity_top_k=3)


c:\Users\caden\Moreland\AutoGen-tuts\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 741/741 [00:09<00:00, 78.74it/s]


In [21]:
# generating query response
response = query_engine.query("how do I join the pregnancy fact table to encounter fact table? It may not be a direct join, so what is the intermediate table? Give me the SQL query to do this.")
print(response)

To join the PregnancyFact table to the EncounterFact table, you need to use the EpisodeKey as the intermediate link. PregnancyFact contains an EpisodeKey column, which links to the EpisodeFact table. The EpisodeFact table, in turn, contains an EncounterKey column that links to the EncounterFact table.

Here is the SQL query to perform this join:

```sql
SELECT
    pf.*,
    ef.*,
    enf.*
FROM
    PregnancyFact pf
    INNER JOIN EpisodeFact ef ON pf.EpisodeKey = ef.EpisodeKey
    INNER JOIN EncounterFact enf ON ef.EncounterKey = enf.EncounterKey
```

This query retrieves data from PregnancyFact, joins it to EpisodeFact using EpisodeKey, and then joins to EncounterFact using EncounterKey.


In [16]:
# show the chunks from the query (Called Nodes for LLama Index)
from textwrap import fill

for node in response.source_nodes:
    print("-" * 80)
    wrapped_text = fill(str(node), width=80)
    print(wrapped_text)

--------------------------------------------------------------------------------
Node ID: c07e14f4-5816-439a-8166-de20b5653648 Text: Each row represents a 5
digit US postal code. Change Type: Non- Snapshot  Columns  PostalCodeKey :
bigint  Surrogate key used to uniquely identify the record  PostalCode :
nvarchar(50)  Postal code ```    ---    ## PregnancyFact    **Extracted:**
2025-07-23 00:35:56 **Content Length:** 9037 characters    **Content:**  ```
General Information  ... Score:  0.530
--------------------------------------------------------------------------------
Node ID: bc1d322e-0c5b-4f88-a0de-0a7a48e81319 Text: Note that this is not the
date the pregnancy began. For historical pregnancy episodes, this is the day
that the pregnancy was added to the patient's obstetric history."        },
{ "ColumnName": "EpisodeEndDateKey",          "DataType": "bigint", "LinksTo":
"DateDim",          "Description": "Date the clinician resolved ... Score:
0.525
--------------------------------

In [17]:
#Print the Chunks from the query with the full text of the chunk
for i, node in enumerate(response.source_nodes):
    # Get basic node metadata
    text = node.node.text
    source_file = node.node.metadata.get("file_name", "Unknown")
    page_number = node.node.metadata.get("page_label", 1)
    score = node.score if hasattr(node, 'score') else None
    
    score_display = f" (Score: {score:.4f})" if score is not None else ""
    print(f"Chunk {i}: {source_file}{score_display}")
    print(f"Text: {text}")
    print(f"Source: {source_file}, Page: {page_number}")
    print("-" * 50)  # Separator between chunks

Chunk 0: epic_documentation_progress.md (Score: 0.5304)
Text: Each row represents a 5 digit US postal code. Change Type: Non-Snapshot
Columns
PostalCodeKey : bigint
Surrogate key used to uniquely identify the record
PostalCode : nvarchar(50)
Postal code
```

---

## PregnancyFact

**Extracted:** 2025-07-23 00:35:56
**Content Length:** 9037 characters

**Content:**
```
General Information
Supported User Types:
Backend Systems and Non-OAuth 2.0
API Type:
Kit Class D Fact
Try It
PregnancyFact
The pregnancy fact contains information about pregnancies. Each row represents a patient's pregnancy. Change Type: Non-Snapshot
Columns
PregnancyKey : bigint
Surrogate key used to uniquely identify the record
PatientKey : bigint
Links to PatientDim
Patient who is pregnant
PatientDurableKey : bigint
Links to PatientDim
Patient who is pregnant
EpisodeKey : bigint
Links to EpisodeFact
Pregnancy episode
WorkingEstimatedDateOfDeliveryKey : bigint
Links to DateDim
Working estimated date of delivery for the

In [9]:
from pprint import pprint